In [3]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
from warnings import warn
import shutil
import json

# custom steps for mobilenetv1
from src.custom_steps import (
    step_mobilenet_streamline,
    step_mobilenet_convert_to_hls_layers,
    step_mobilenet_convert_to_hls_layers_separate_th,
    step_mobilenet_lower_convs,
    step_mobilenet_slr_floorplan,
    custom_step_partition,
)

def multi_build(model_file,final_output_dir):
# model_file = "/home/pgeel/FINNv0.8.1_repo/build_KV260/finn/notebooks/MSC_Git_files/mobilenet_v1/models/mobilenet_streamline.onnx"
# final_output_dir = "half_baseline_build"

    folding_config_file = "/home/pgeel/FINNv0.8.1_repo/build_KV260/finn/notebooks/MSC_Git_files/mobilenet_v1/folding_config/auto_build_folding.json"
    #Delete previous run results if exist
    if os.path.exists(final_output_dir):
        shutil.rmtree(final_output_dir)
        # os.remove(final_output_dir)
        print("Previous run results deleted!")

    mobilenet_build_steps =  [
    #     step_mobilenet_streamline,
        # custom_step_partition,
        step_mobilenet_lower_convs,
        step_mobilenet_convert_to_hls_layers_separate_th,
        "step_create_dataflow_partition",
        "step_apply_folding_config",
        "step_generate_estimate_reports",
        "step_hls_codegen",
        "step_hls_ipgen",
        "step_set_fifo_depths",
        "step_create_stitched_ip",
        "step_synthesize_bitfile",
        "step_make_pynq_driver",
        "step_deployment_package",
    ]

    cfg = build.DataflowBuildConfig(
        output_dir          = final_output_dir,
        steps               = mobilenet_build_steps,
        mvau_wwidth_max     = 80,
        target_fps          = 25,
        synth_clk_period_ns = 10.0,
        folding_config_file = folding_config_file,
        board               = "KV260_SOM",
        shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
        generate_outputs=[
            build_cfg.DataflowOutputType.BITFILE,
            build_cfg.DataflowOutputType.PYNQ_DRIVER,
            build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
            build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
        ]
    )
#     try:    
#         build.build_dataflow_cfg(m, cfg)    
#     except:
#         pass
    return model_file, cfg

In [4]:
import os,json
import time
outdir = "testing"
if not os.path.exists(outdir):
    os.mkdir(outdir)
    
builds = []
for f in os.listdir("CPU_testing"):
    if os.path.isdir(os.path.join("CPU_testing", f)):
        model_file = os.path.join("CPU_testing", f,"partition_0.onnx")
        m,cfg = multi_build(model_file,os.path.join(outdir,f))
        try: 
            build.build_dataflow_cfg(m, cfg)    
        except:
            print("Stopped at %s as it did not fit on the FPGA!" %m)
            sys.exit(1)


Building dataflow accelerator from CPU_testing/MultiThreshold_0_out0/partition_0.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in testing/MultiThreshold_0_out0
Build log is at testing/MultiThreshold_0_out0/build_dataflow.log
Running step: step_mobilenet_lower_convs [1/12]
Running step: step_mobilenet_convert_to_hls_layers_separate_th [2/12]
Running step: step_create_dataflow_partition [3/12]
Running step: step_apply_folding_config [4/12]
Running step: step_generate_estimate_reports [5/12]
Running step: step_hls_codegen [6/12]
Running step: step_hls_ipgen [7/12]
Running step: step_set_fifo_depths [8/12]
Running step: step_create_stitched_ip [9/12]
Running step: step_synthesize_bitfile [10/12]
Running step: step_make_pynq_driver [11/12]
Running step: step_deployment_package [12/12]
Completed successfully
Building dataflow accelerator from CPU_testing/MultiThreshold_1_out0/partition_0.onnx
Intermediate outputs will be generated in /works

Running step: step_hls_ipgen [7/12]
Running step: step_set_fifo_depths [8/12]
Running step: step_create_stitched_ip [9/12]
Running step: step_synthesize_bitfile [10/12]
Running step: step_make_pynq_driver [11/12]
Running step: step_deployment_package [12/12]
Completed successfully
Building dataflow accelerator from CPU_testing/MultiThreshold_10_out0/partition_0.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in testing/MultiThreshold_10_out0
Build log is at testing/MultiThreshold_10_out0/build_dataflow.log
Running step: step_mobilenet_lower_convs [1/12]
Running step: step_mobilenet_convert_to_hls_layers_separate_th [2/12]
Running step: step_create_dataflow_partition [3/12]
Running step: step_apply_folding_config [4/12]
Running step: step_generate_estimate_reports [5/12]
Running step: step_hls_codegen [6/12]
Running step: step_hls_ipgen [7/12]
Running step: step_set_fifo_depths [8/12]
Running step: step_create_stitched_ip [9/12]
Running 

Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/workspace/finn/src/finn/builder/build_dataflow_steps.py", line 632, in step_synthesize_bitfile
    model = model.transform(
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/workspace/finn/src/finn/transformation/fpgadataflow/make_zynq_proj.py", line 364, in apply
    model = model.transform(
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/workspace/finn/src/finn/transformation/fpgadataflow/make_zynq_proj.py", line 276, in apply
    raise Exception(
Exception: Synthesis failed, no bitfile found. Check logs under /workspace/results/vivado_zynq_proj_xb0dkvve


> /workspace/finn/src/finn/transformation/fpgadataflow/make_zynq_proj.py(276)apply()
    274         )
    275         if not os.path.isfile(bitfile_name):
--> 276             raise Exception(
    277                 "Synthesis failed, no bitfile found. Check logs under %s"
    278                 % vivado_pynq_proj_dir

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
Build failed
Building dataflow accelerator from CPU_testing/MultiThreshold_17_out0/partition_0.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in testing/MultiThreshold_17_out0
Build log is at testing/MultiThreshold_17_out0/build_dataflow.log
Running step: step_mobilenet_lower_convs [1/12]
Running step: step_mobilenet_convert_to_hls_layers_separate_th [2/12]
Running step: step_create_dataflow_partition [3/12]
Running step: step_apply_folding_config [4/12]
Running step: step_generate_estimate_reports [5/12]
Running step: step_hls_codegen [6/12]


Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/workspace/finn/src/finn/builder/build_dataflow_steps.py", line 426, in step_hls_codegen
    model = model.transform(
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/workspace/finn/src/finn/transformation/fpgadataflow/prepare_ip.py", line 88, in apply
    _codegen_single_node(node, model, self.fpgapart, self.clk)
  File "/workspace/finn/src/finn/transformation/fpgadataflow/prepare_ip.py", line 55, in _codegen_single_node
    inst.code_generation_ipgen(model, fpgapart, clk)
  File "/workspace/finn/src/finn/custom_op/fpgadataflow/hlscustomop.py", line 271, in code_generation_ipgen
    self.generate_params(model, path)
  File "/workspace/finn/src/finn/custom_op/fpgadataflow/matrixvectoractiva

...

> /workspace/qonnx/src/qonnx/core/modelwrapper.py(336)find_producer()
    334         """Finds and returns the node that produces the tensor with given name."""
    335         for x in self._model_proto.graph.node:
--> 336             if tensor_name in x.output:
    337                 return x
    338         return None

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
Build failed
Building dataflow accelerator from CPU_testing/MultiThreshold_25_out0/partition_0.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in testing/MultiThreshold_25_out0
Build log is at testing/MultiThreshold_25_out0/build_dataflow.log
Running step: step_mobilenet_lower_convs [1/12]


Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/shares/bulk/pgeel/FINNv0.8.1_repo/build_KV260/finn/notebooks/MSC_Git_files/mobilenet_v1/src/custom_steps.py", line 98, in step_mobilenet_lower_convs
    model = model.transform(LowerConvsToMatMul())
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/workspace/qonnx/src/qonnx/transformation/lower_convs_to_matmul.py", line 144, in apply
    model.set_initializer(weight_name, W_matmul)
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 272, in set_initializer
    tensor_init_proto = np_helper.from_array(tensor_value)
  File "/home/pgeel/.local/lib/python3.8/site-packages/onnx/numpy_helper.py", line 134, in from_array
    tensor.raw_data = arr.tobytes()  # note: tobytes() is only aft

> /home/pgeel/.local/lib/python3.8/site-packages/onnx/numpy_helper.py(134)from_array()
    132             "Numpy data type not understood yet: {}".format(str(arr.dtype)))
    133     tensor.data_type = dtype
--> 134     tensor.raw_data = arr.tobytes()  # note: tobytes() is only after 1.9.
    135     if sys.byteorder == 'big':
    136         # Convert endian from big to little

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
Build failed
Building dataflow accelerator from CPU_testing/MultiThreshold_26_out0/partition_0.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in testing/MultiThreshold_26_out0
Build log is at testing/MultiThreshold_26_out0/build_dataflow.log
Running step: step_mobilenet_lower_convs [1/12]


Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/shares/bulk/pgeel/FINNv0.8.1_repo/build_KV260/finn/notebooks/MSC_Git_files/mobilenet_v1/src/custom_steps.py", line 98, in step_mobilenet_lower_convs
    model = model.transform(LowerConvsToMatMul())
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/workspace/qonnx/src/qonnx/transformation/lower_convs_to_matmul.py", line 144, in apply
    model.set_initializer(weight_name, W_matmul)
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 272, in set_initializer
    tensor_init_proto = np_helper.from_array(tensor_value)
  File "/home/pgeel/.local/lib/python3.8/site-packages/onnx/numpy_helper.py", line 134, in from_array
    tensor.raw_data = arr.tobytes()  # note: tobytes() is only aft

> /home/pgeel/.local/lib/python3.8/site-packages/onnx/numpy_helper.py(134)from_array()
    132             "Numpy data type not understood yet: {}".format(str(arr.dtype)))
    133     tensor.data_type = dtype
--> 134     tensor.raw_data = arr.tobytes()  # note: tobytes() is only after 1.9.
    135     if sys.byteorder == 'big':
    136         # Convert endian from big to little

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
Build failed
